# Schneider Electrick Hackaton - Data Science
## Team 4 - Gerard Jover Pujol - Pol Gràcia Espelt

### Modules import

In [28]:
import os
import sklearn
import pandas as pd
import numpy as np 
import requests as r
import pandas_profiling
import pdfplumber
import sklearn.model_selection

### Data load

In [2]:
project_path = "./"
train_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'train' in x]
test_data_dir_csv = [project_path + 'Data/CSV/' + x for x in os.listdir(project_path+'Data/CSV') if 'test' in x][0]
train_data_dir_pdf = [project_path + 'Data/pdf/' + x for  x in os.listdir(project_path+'Data/pdf')]
train_data_api = project_path + 'Data/json/links.txt'

#### CSV

In [3]:
train_csv = pd.concat([pd.read_csv(train_data_dir_csv[0]), pd.read_csv(train_data_dir_csv[1], sep = ';')])

In [4]:
train_csv.describe()

,reportingYear,MONTH,DAY,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,DAY WITH FOGS
count,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000,37127.000000
mean,2012.951841,6.487031,14.540335,15.510519,18.013596,22.523559,9.452422,10.441560,13.431015,2.228217
std,3.856172,3.467030,8.128336,3.070188,2.313407,3.064919,5.232214,5.094000,5.225196,3.775751
min,2007.000000,1.000000,1.000000,8.062689,14.000287,15.055647,-3.141464,-0.199176,0.894827,0.000000
25%,2010.000000,3.000000,8.000000,13.315477,16.007158,20.361339,5.855159,7.158428,9.866841,0.000000
50%,2013.000000,7.000000,15.000000,15.503568,18.006838,22.553985,9.734382,10.733914,13.723792,1.000000
75%,2016.000000,9.000000,22.000000,17.708038,20.013530,24.713992,13.287165,14.162492,17.235091,2.000000
max,2020.000000,12.000000,28.000000,22.991382,21.999973,29.914367,20.855885,19.999403,24.855425,19.000000


### api

In [5]:
with open(train_data_api) as file:
    links = [link.strip() for link in file]

int_cols = ['MONTH','reportingYear', 'DAY', 'max_wind_speed', 'avg_wind_speed', 'min_wind_speed', 'max_temp', 'min_temp', 'avg_temp', 'DAY WITH FOGS']
json_df = pd.DataFrame()
for link in links:
    data_json = r.get(link).json()
    tmp_df = pd.json_normalize(data_json)
    #set index
    tmp_df.set_index(tmp_df[''], inplace = True)
    tmp_df.drop('', axis = 1, inplace=True)
    #values to int
    tmp_df[int_cols] = tmp_df[int_cols].apply(pd.to_numeric)
    #print(tmp_df.columns)
    #concat
    json_df = pd.concat([json_df, tmp_df])
    #train_csv = pd.concat([train_csv, tmp_df])

### Combine csv and json

In [6]:
df_bo = pd.concat([train_csv, json_df])

### Replace nans

In [7]:
label2code = pd.DataFrame(df_bo[['EPRTRAnnexIMainActivityLabel', 'EPRTRAnnexIMainActivityCode', 'EPRTRSectorCode']].value_counts()).reset_index()

In [8]:
label2SC = dict(zip(list(label2code.EPRTRAnnexIMainActivityLabel), list(label2code.EPRTRSectorCode)))
label2MAC = dict(zip(list(label2code.EPRTRAnnexIMainActivityLabel), list(label2code.EPRTRAnnexIMainActivityCode)))

In [9]:
f = lambda x: label2SC[x] if x != 'Chemical installations for the production on an industrial scale of basic organic chemicals: Organometallic compounds' else np.nan
df_bo['EPRTRSectorCode'] = df_bo['EPRTRAnnexIMainActivityLabel'].apply(f)

In [10]:
f = lambda x: label2MAC[x] if x != 'Chemical installations for the production on an industrial scale of basic organic chemicals: Organometallic compounds' else np.nan
df_bo['EPRTRAnnexIMainActivityCode'] = df_bo['EPRTRAnnexIMainActivityLabel'].apply(f)

### pdf

In [11]:
def pdf_2_dict(pdf_path):
    # Extract text from pdf file
    with pdfplumber.open(pdf_path) as temp:
        first_page = temp.pages[0]
        text = first_page.extract_text()
        text_list = text.split("\n")

    # Create empty dict
    pdf_dict = {}

    # Generate new fields
    pdf_dict["id"] = int(text_list[1].split()[1])
    pdf_dict["facilityName"] = " ".join(text_list[2].split()[2:-1])
    pdf_dict["FacilityInspireID"] = text_list[3].split()[1]
    pdf_dict["countryName"] = " ".join(text_list[4].split("CONTINENT:")[0].split()[1:])
    pdf_dict["CONTINENT"] = " ".join(text_list[4].split("CONTINENT:")[1:])[1:]
    pdf_dict["City"] = " ".join(text_list[5].split()[1:])
    pdf_dict["EPRTRSectorCode"] = int(" ".join(text_list[6].split("eprtrSectorName:")[0].split()[1:]))
    pdf_dict["eprtrSectorName"] = " ".join(text_list[6].split("eprtrSectorName:")[1:])[1:]
    pdf_dict["EPRTRAnnexIMainActivityCode"] = text_list[7].split(":")[1]
    pdf_dict["targetRelease"] = " ".join(text_list[8].split("pollutant:")[0].split()[1:])
    pdf_dict["pollutant"] = " ".join(text_list[8].split("pollutant:")[1:])[1:]
    pdf_dict["emissions"] = float(text_list[9].split()[1].replace(",", "."))
    pdf_dict["DAY"] = int(text_list[10].split()[1])
    pdf_dict["MONTH"] = int(text_list[10].split()[3])
    pdf_dict["reportingYear"] = int(text_list[10].split()[5])
    pdf_dict["max_wind_speed"] = float(text_list[12].split()[1].replace(",", ".")) / 10e15
    pdf_dict["min_wind_speed"] = float(text_list[12].split()[3].replace(",", "."))/ 10e15
    pdf_dict["avg_wind_speed"] = float(text_list[12].split()[5].replace(",", "."))/ 10e15
    pdf_dict["max_temp"] = float(text_list[13].split()[1].replace(",", ".")) / 10e15
    pdf_dict["min_temp"] = float(text_list[13].split()[3].replace(",", ".")) / 10e15
    pdf_dict["avg_temp"] = float(text_list[13].split()[5].replace(",", "."))/ 10e15
    pdf_dict["DAY WITH FOGS"] = int(text_list[14].split()[-1])
    pdf_dict["REPORTER NAME"] = text_list[15].split("REPORTER NAME:")[-1]
    pdf_dict["CITY ID"] = text_list[16].split("CITY_ID ")[-1]

    return pdf_dict

In [12]:
df_pdf = pd.DataFrame()
for pdf_path in train_data_dir_pdf:
    pdf_dict = pdf_2_dict(pdf_path)
    pd_series = pd.Series(pdf_dict)
    df_pdf = df_pdf.append(pd_series, ignore_index=True)

### combine csv, json and pdf

In [13]:
df_final = pd.concat([df_bo, df_pdf])

In [14]:
df_final.shape

(65710, 25)

### remove columns

In [15]:
drop_cols = ["id", "emissions", "EPRTRAnnexIMainActivityLabel", "targetRelease", "CONTINENT", "CITY ID", 'max_wind_speed',	'avg_wind_speed',	'min_wind_speed',	'max_temp',	'avg_temp',	'min_temp']
df_final = df_final.drop(drop_cols, axis=1)

### dtypes

In [16]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [17]:
for col in df_final.columns:
    if col != "pollutant":
        if is_string_dtype(df_final[col]):
            df_final[col] = df_final[col].astype('category').cat.codes
        if is_numeric_dtype(df_final[col]):
            df_final[col] = df_final[col].astype(np.int64)

### target

In [18]:
from sklearn.preprocessing import OrdinalEncoder


ord_enc = OrdinalEncoder()
df_final["pollutant"] = ord_enc.fit_transform(df_final[["pollutant"]])
#ord_enc.inverse_transform

In [ ]:
# df_final.to_csv("df_final.csv")

### Models

In [32]:
target = 'pollutant'
X = df_final.drop(target, axis = 1).to_numpy()
y = df_final[target].to_numpy()
df_final.to_numpy()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.20, random_state=42)

In [34]:
from modelHandler import ModelHandler

In [35]:
xgb = ModelHandler(X = X_train, Y = y_train, model = 'XGB')

In [ ]:
xgb.fit()

### Training

### Testing